In [29]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define the path to the folder
folder_path = os.path.join("datasets", "si02", "")
os.makedirs(folder_path, exist_ok=True)
# Get a list of all items in the folder
all_items = os.listdir(folder_path)

# Filter the list to include only files
files = [item for item in all_items if item.endswith('.txt')]

dataFrame = pd.DataFrame()


for i in files:
    dataHelper = pd.read_csv(folder_path + i, sep='\t', header=None, index_col=False)
    info = i.split('_')
    dataType = info[1]
    thickness = info[2].replace('-', '.')
    dataHelper = dataHelper.drop(index = [0])
    dataHelper = dataHelper.drop(columns = [4])
    dataHelper.columns = ['wavelength', str(dataType) + '60', str(dataType) + '65', str(dataType) + '70']
    dataHelper['thickness'] = thickness

    
    for j in files:
        jDataHelper = pd.read_csv(folder_path + j, sep='\t', header=None, index_col=False)
        jInfo = j.split('_')
        jDataType = jInfo[1]
        jThickness = jInfo[2].replace('-', '.')
        jDataHelper = jDataHelper.drop(index = [0])
        jDataHelper = jDataHelper.drop(columns = [4])
        
        if dataType != jDataType and thickness == jThickness:
            jDataHelper.columns = ['wavelength', str(jDataType) + '60', str(jDataType) + '65', str(jDataType) + '70']
            dataHelper[str(jDataType) + '60'] = jDataHelper[str(jDataType) + '60']
            dataHelper[str(jDataType) + '65'] = jDataHelper[str(jDataType) + '65']
            dataHelper[str(jDataType) + '70'] = jDataHelper[str(jDataType) + '70']
            dataHelper.to_csv(folder_path + thickness + '.csv', index=False)



In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.signal import find_peaks
import numpy as np


files = [item for item in all_items if item.endswith('.csv')]

newDataFrame = pd.DataFrame()

for i in files:
    dataHelper = pd.read_csv(folder_path + i, sep=',')
    dataHelper.columns = ['wavelength', 'psi60', 'psi65', 'psi70','thickness','delta60','delta65','delta70']
    newDataFrame = pd.concat([newDataFrame, dataHelper], ignore_index=True)

newDataFrame.head()

,wavelength,psi60,psi65,psi70,thickness,delta60,delta65,delta70
0,400.0,26.521,21.973,16.974,11.4,154.00,142.54,119.89
1,410.0,25.474,20.634,15.559,11.4,154.64,142.88,118.59
2,420.0,24.660,19.631,14.416,11.4,155.29,143.23,117.56
3,430.0,23.939,18.762,13.524,11.4,155.83,143.53,116.36
4,440.0,23.306,17.990,12.686,11.4,156.44,144.30,115.05


In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from scipy.signal import find_peaks
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
folder_path = os.path.join("datasets", "si02", "")
os.makedirs(folder_path, exist_ok=True)
# Get a list of all items in the folder
all_items = os.listdir(folder_path)

# Filter the list to include only files
files = [item for item in all_items if item.endswith('.csv')]

polyFeatures = PolynomialFeatures(degree=6)
polyReg = LinearRegression()
delteFeature = PolynomialFeatures(degree=6)
deltaReg = LinearRegression()
newDataFrame = pd.DataFrame()
for i in files:
    dataHelper = pd.read_csv(folder_path + i, sep=',')
    dataHelper.columns = ['wavelength', 'psi60', 'psi65', 'psi70','thickness','delta60','delta65','delta70']
    T = dataHelper['thickness'].values[1]
    psi65 =dataHelper['psi65']
    wv = dataHelper['wavelength']
    vwPoly = polyFeatures.fit_transform(wv.values.reshape(-1,1))
    del65 = dataHelper['delta65']
    polyReg.fit(vwPoly, psi65)
    deltaReg.fit(vwPoly, del65)
    a1 = polyReg.coef_[1]
    a2 = polyReg.coef_[2]
    a3 = polyReg.coef_[3]
    a4 = polyReg.coef_[4]
    a5 = polyReg.coef_[5]
    a6 = polyReg.coef_[6]
    d1 = deltaReg.coef_[1]
    d2 = deltaReg.coef_[2]
    d3 = deltaReg.coef_[3]
    d4 = deltaReg.coef_[4]
    d5 = deltaReg.coef_[5]
    d6 = deltaReg.coef_[6]
    peaksPsi = peaks, _ = find_peaks(dataHelper['psi65'])
    peaksDel = peaks, _ = find_peaks(dataHelper['delta65'])
    data = pd.Series([a1,a2,a3,a4,a5,a6,d1,d2,d3,d4,d5,d6, polyReg.intercept_,deltaReg.intercept_,len(peaksPsi),len(peaksDel),T])
    df = data.to_frame().T
    newDataFrame = pd.concat([newDataFrame, df], ignore_index=True)

newDataFrame.columns = ['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel', 'T']


print(newDataFrame)


             a1            a2        a3            a4            a5  \
0 -1.022153e-11 -4.293742e-09 -0.000001  3.039658e-09 -3.020801e-12   
1 -1.018628e-11 -4.278938e-09 -0.000001  3.021210e-09 -2.996578e-12   
2 -1.364278e-11 -7.153055e-09 -0.000002  4.701720e-09 -4.757241e-12   
3 -9.996893e-12 -4.199381e-09 -0.000001  2.988517e-09 -2.980815e-12   
4 -9.854277e-12 -4.139473e-09 -0.000001  2.942548e-09 -2.931673e-12   
5 -9.887474e-12 -4.153418e-09 -0.000001  2.829269e-09 -2.735057e-12   
6 -9.905790e-12 -4.161112e-09 -0.000001  2.958517e-09 -2.948780e-12   
7 -1.005282e-11 -4.222875e-09 -0.000001  3.001613e-09 -2.990786e-12   
8 -1.002856e-11 -4.212684e-09 -0.000001  2.993208e-09 -2.982256e-12   
9 -1.008324e-11 -4.235653e-09 -0.000001  3.001819e-09 -2.985328e-12   

             a6            d1            d2            d3            d4  \
0  1.037991e-15  2.353206e-12  9.885082e-10  2.502564e-07 -6.020007e-10   
1  1.028098e-15  2.065164e-12  8.675109e-10  2.196240e-07 -5.013437e

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

coefficients = newDataFrame[['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel']]
coefficients = MinMaxScaler().fit_transform(coefficients)
tScaler = MinMaxScaler()
newYT = newDataFrame['T']
YTscaled = tScaler.fit_transform(newYT.values.reshape(-1,1))



newXT_train, newXT_test, newYT_train, newYT_test = train_test_split(coefficients, YTscaled, test_size=0.2, random_state=0)


regT = LinearRegression()


regT.fit(newXT_train, newYT_train)

print("RMSE dla T: ", mean_squared_error(newYT_test, regT.predict(newXT_test)))
print("wariancja dla T: ", newYT_test.var())


RMSE dla T:  40.93017936220721
wariancja dla T:  0.07141795481569563
